# Relay Example Notebook (Local Development)

This notebook demonstrates how to use Relay to:
1. Submit batch jobs
2. Track job progress
3. Retrieve results from old jobs

The key feature of Relay is that all jobs and results are stored in a workspace directory, so you can access them across sessions.

**This version uses the local codebase instead of the pip-installed package, making it ideal for testing local changes.**

## Setup

This notebook automatically adds the parent directory to Python's path so it can import from the local `relay` package. No installation needed!


In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
import sys
import time
from pathlib import Path

# Add the parent directory to Python path to import from local codebase
# This allows importing from the local 'relay' package without installing it
notebook_dir = Path.cwd()
# If we're in the relay directory (where the notebook is), add the parent
# Otherwise, if we're in a parent directory, add the current directory
if (notebook_dir / "relay").exists():
    # We're in the root directory, add it to path
    project_root = notebook_dir
elif notebook_dir.name == "relay":
    # We're inside the relay package directory, go up one level
    project_root = notebook_dir.parent
else:
    # Try to find the relay directory by going up
    project_root = notebook_dir
    while project_root.parent != project_root:
        if (project_root / "relay").exists():
            break
        project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✓ Added {project_root} to Python path")

# Import Relay from local codebase
try:
    from relay import RelayClient, BatchRequest, __version__
    print(f"✓ Relay version {__version__} imported successfully from local codebase")
except ImportError as e:
    print("✗ Error: Could not import relay from local codebase.")
    print(f"   Error: {e}")
    print(f"   Current directory: {os.getcwd()}")
    print(f"   Python path: {sys.path[:3]}")
    raise

# Check if API key is set
api_key_path = "TOGETHER_API_KEY"
assert os.getenv("TOGETHER_API_KEY")


✓ Relay version 0.1.0 imported successfully from local codebase


## Part 1: Submitting Jobs

Create a workspace and submit some batch jobs.


In [21]:
# Create a workspace - all jobs will be stored here
workspace_dir = "example_workspace"
relay = RelayClient(directory=workspace_dir)

print(f"✓ Created workspace: {workspace_dir}")


✓ Created workspace: example_workspace


In [22]:
# Create some batch requests
requests = [
    BatchRequest(
        id="req-1",
        model="openai/gpt-oss-20b",
        system_prompt="You are a helpful assistant.",
        prompt="What is 2+2?",
        provider_args={"reasoning_effort": "medium", "temperature": 0.6}
    ),
    BatchRequest(
        id="req-2",
        model="openai/gpt-oss-20b",
        system_prompt="You are a helpful assistant.",
        prompt="What is the capital of France?",
        provider_args={"reasoning_effort": "medium", "temperature": 0.6}
    ),
    BatchRequest(
        id="req-3",
        model="openai/gpt-oss-20b",
        system_prompt="You are a helpful assistant.",
        prompt="Explain quantum computing in one sentence.",
        provider_args={"reasoning_effort": "medium", "temperature": 0.6}
    ),
]

print(f"Created {len(requests)} requests")

Created 3 requests


In [23]:
# Submit the batch job with a unique ID
job_id = "notebook-demo-001-together-oss-20b-medium-reasoning"

job = relay.submit_batch(
    requests=requests,
    job_id=job_id,
    provider="together",
    description="Notebook demonstration batch job"
)

print(f"✓ Job submitted successfully!")
print(f"  Job ID: {job.job_id}")
print(f"  Submitted at: {job.submitted_at}")
print(f"  Status: {job.status}")
print(f"  Number of requests: {job.n_requests}")


Uploading file requests.jsonl: 100%|███████████████████████████████████████████████████████████████████████████████████| 903/903 [00:00<00:00, 1.32kB/s]


✓ Job submitted successfully!
  Job ID: notebook-demo-001-together-oss-20b-medium-reasoning
  Submitted at: 2025-12-20 14:27:43.863635
  Status: BatchJobStatus.VALIDATING
  Number of requests: 3


## Part 2: Tracking Jobs

List all jobs in the workspace and monitor their progress.


In [24]:
# List all jobs in the workspace
all_jobs = relay.list_jobs()
print(f"Found {len(all_jobs)} job(s) in workspace:")
for jid in all_jobs:
    print(f"  - {jid}")


Found 2 job(s) in workspace:
  - notebook-demo-001-together-oss-20b
  - notebook-demo-001-together-oss-20b-medium-reasoning


In [25]:
# Get detailed information about a specific job
job_info = relay.get_job(job_id)
if job_info:
    print(f"Job: {job_info['job_id']}")
    print(f"  Description: {job_info['description']}")
    print(f"  Provider: {job_info['provider']}")
    print(f"  Status: {job_info['status']}")
    print(f"  Submitted: {job_info['submitted_at']}")
    print(f"  Requests: {job_info['n_requests']}")
    print(f"  Completed: {job_info.get('completed_requests', 0)}")
    print(f"  Failed: {job_info.get('failed_requests', 0)}")


Job: notebook-demo-001-together-oss-20b-medium-reasoning
  Description: Notebook demonstration batch job
  Provider: together
  Status: VALIDATING
  Submitted: 2025-12-20T14:27:43.863635
  Requests: 3
  Completed: 0
  Failed: 0


In [28]:
# Monitor job progress
job_status = relay.monitor_batch(job_id)
display(job_status)


BatchJob(job_id='notebook-demo-001-together-oss-20b-medium-reasoning', submitted_at=datetime.datetime(2025, 12, 20, 19, 27, 42, 374110, tzinfo=TzInfo(UTC)), status=<BatchJobStatus.COMPLETED: 'COMPLETED'>, n_requests=3, completed_requests=0, failed_requests=0)

## Part 3: Getting Results

Next, we can get the results from this job.


In [29]:
results = relay.retrieve_batch_results(job_id)
    
print(f"\n✓ Retrieved {len(results)} results")
print(f"\nResults are automatically saved to: {workspace_dir}/{job_id}_results.json")

# Display sample results
print("\n" + "=" * 60)
print("Sample Results:")
print("=" * 60)

for i, result in enumerate(results[:3], 1):  # Show first 3
    print(f"\nResult {i}:")
    custom_id = result.get('custom_id', 'N/A')
    print(f"  Request ID: {custom_id}")
    
    # Extract response based on OpenAI format
    if 'response' in result:
        response = result['response']
        if 'body' in response:
            body = response['body']
            if 'output' in body:
                output = body['output']
                # Truncate long outputs
                output_str = str(output)
                if len(output_str) > 200:
                    output_str = output_str[:200] + "..."
                print(f"  Output: {output_str}")
    
    if 'error' in result:
        print(f"  Error: {result['error']}")

if len(results) > 3:
    print(f"\n  ... and {len(results) - 3} more results")



✓ Retrieved 3 results

Results are automatically saved to: example_workspace/notebook-demo-001-together-oss-20b-medium-reasoning_results.json

Sample Results:

Result 1:
  Request ID: req-1

Result 2:
  Request ID: req-2

Result 3:
  Request ID: req-3


## Part 4: Getting Results from Old Jobs

We can also get results from old jobs.


In [16]:
new_relay = RelayClient(directory=workspace_dir)
print("✓ New RelayClient created")

# All jobs are still accessible!
existing_jobs = new_relay.list_jobs()
print(f"\nFound {len(existing_jobs)} existing job(s):")
for jid in existing_jobs:
    print(f"  - {jid}")


✓ New RelayClient created

Found 1 existing job(s):
  - notebook-demo-001-together-oss-20b


In [17]:
# Check if results exist for a job
if new_relay.has_results(job_id):
    print(f"✓ Results exist for {job_id}")
else:
    print(f"✗ No results found for {job_id}")
    print("  (Results are saved when you call retrieve_batch_results)")


✓ Results exist for notebook-demo-001-together-oss-20b


In [18]:
results = new_relay.retrieve_batch_results(job_id)
    
print(f"\n✓ Retrieved {len(results)} results")
print(f"\nResults are automatically saved to: {workspace_dir}/{job_id}_results.json")

# Display sample results
print("\n" + "=" * 60)
print("Sample Results:")
print("=" * 60)

for i, result in enumerate(results[:3], 1):  # Show first 3
    print(f"\nResult {i}:")
    custom_id = result.get('custom_id', 'N/A')
    print(f"  Request ID: {custom_id}")
    
    # Extract response based on OpenAI format
    if 'response' in result:
        response = result['response']
        if 'body' in response:
            body = response['body']
            if 'output' in body:
                output = body['output']
                # Truncate long outputs
                output_str = str(output)
                if len(output_str) > 200:
                    output_str = output_str[:200] + "..."
                print(f"  Output: {output_str}")
    
    if 'error' in result:
        print(f"  Error: {result['error']}")

if len(results) > 3:
    print(f"\n  ... and {len(results) - 3} more results")



✓ Retrieved 3 results

Results are automatically saved to: example_workspace/notebook-demo-001-together-oss-20b_results.json

Sample Results:

Result 1:
  Request ID: req-1

Result 2:
  Request ID: req-2

Result 3:
  Request ID: req-3
